# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [1]:
import main

**Zadanie 1**

Znajdź liczbę wszystkich filmów o tej samej długości.

In [2]:
df = main.zadanie_1()
df

,length,count
0,46,5
1,47,7
2,48,11
3,49,5
4,50,9
...,...,...
135,181,10
136,182,6
137,183,5
138,184,8


**Zadanie 2**

Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [3]:
df = main.zadanie_2()
df

,city,first_name,last_name
0,A Corua (La Corua),Julie,Sanchez
1,Abha,Peggy,Myers
2,Abu Dhabi,Tom,Milner
3,Acua,Glen,Talbert
4,Adana,Larry,Thrasher
...,...,...,...
594,Zaria,Constance,Reid
595,Zeleznogorsk,Jack,Foust
596,Zhezqazghan,Byron,Box
597,Zhoushan,Guy,Brownlee


**Zadanie 3**

Oblicz średni koszt wypożyczenia wszystkich filmów.

In [4]:
df = main.zadanie_3()
df

,average_rental_rate
0,2.98


**Zadanie 4**

Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [5]:
df = main.zadanie_4()
df

,category,count
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


**Zadanie 5**

Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [6]:
df = main.zadanie_5()
df

,first_name,last_name,country
0,Vera,Mccoy,Afghanistan
1,June,Carroll,Algeria
2,Mario,Cheatham,Algeria
3,Judy,Gray,Algeria
4,Anthony,Schwab,American Samoa
...,...,...,...
594,Ella,Oliver,Yemen
595,William,Satterfield,Yemen
596,Maria,Miller,Yugoslavia
597,Max,Pitt,Yugoslavia


**Zadanie 6**

Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [7]:
df = main.zadanie_6()
df

,store_id,manager_staff_id,address_id,last_update
0,2,2,2,2006-02-15 09:57:12


**Zadanie 7**

Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [8]:
df = main.zadanie_7()
df

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Clara,Shaw,4808
595,Wesley,Bull,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


**Zadanie 8**

Oblicz średnią wartość wypożyczenia filmu.

In [9]:
df = main.zadanie_8()
df

,title,average_amount
0,Frontier Cabin,5.990000
1,Arachnophobia Rollercoaster,4.772609
2,Cruelty Unforgiven,1.156667
3,Intolerable Intentions,5.704286
4,Monsoon Cause,6.101111
...,...,...
953,Reef Salute,2.171818
954,Bugsy Song,5.101111
955,Orient Closer,4.754706
956,Brotherhood Blanket,4.294348


**Zadanie 9**

Oblicz średnią wartość długości filmu we wszystkich kategoriach

In [10]:
df = main.zadanie_9()
df

,category,average_length
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


**Zadanie 10**

Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [11]:
df = main.zadanie_10()
df

,category,max_title_length_per_category
0,Horror,27
1,Comedy,23
2,Sci-Fi,20
3,Drama,22
4,Foreign,20
5,Classics,23
6,Games,21
7,New,21
8,Travel,23
9,Music,22


**Zadanie 11**

Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [12]:
df = main.zadanie_11()
df

,category,max_length
0,Horror,181
1,Comedy,185
2,Sci-Fi,185
3,Drama,181
4,Foreign,184
5,Classics,184
6,Games,185
7,New,183
8,Travel,185
9,Music,185


In [13]:
df = main.film_in_category(1)
df

,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
...,...,...,...
59,Uprising Uptown,English,Action
60,Waterfront Deliverance,English,Action
61,Werewolf Lola,English,Action
62,Women Dorado,English,Action


In [14]:
df = main.number_films_in_category(1)
df

,category,count
0,Action,64


In [15]:
df = main.number_film_by_length(45, 48)
df

,length,count
0,46,5
1,47,7
2,48,11


In [16]:
df = main.client_from_city('Athenai')
df

,city,first_name,last_name
0,Athenai,Linda,Williams


In [17]:
df = main.avg_amount_by_length(48)
df

,length,avg
0,48,4.295389


In [18]:
df = main.client_by_sum_length(2000)
df

,first_name,last_name,sum
0,Eileen,Carr,2031
1,Kenneth,Gooden,2033
2,Wayne,Truong,2044
3,Floyd,Gandy,2063
4,Paula,Bryant,2084
...,...,...,...
574,Wesley,Bull,4808
575,Clara,Shaw,4808
576,Tammy,Sanders,5065
577,Eleanor,Hunt,5360


In [19]:
df = main.category_statistic_length('Action')
df

,category,avg,sum,min,max
0,Action,111.609375,7143,47,185
